In [1]:
import numpy
import scipy.stats
from scipy.integrate import quad

In [2]:
CHEB_NODES_NB = 15

In [3]:
N = 2

# Gaussian expectation vector
Mu = numpy.zeros(N)

# Gaussian variance-covariance matrix
rho = 0.
Sigma = numpy.array([[1., rho], [rho, 1.]])

print "Mu = %s" % Mu
print "Sigma = \n %s" % Sigma

ctp = 0.

Mu = [ 0.  0.]
Sigma = 
 [[ 1.  0.]
 [ 0.  1.]]


# Link between paper and code

\begin{equation*}
\ell(x) = \sum_k x_k + \frac{1}{2} \sum_k (x_k^+)^2
\end{equation*}

We need to solve the following system:
    
\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ \mathbb{E} \left( \left(X_k - m_k^* \right)^+ \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \mathbb{E} \left( X_k - m_k^* \right) 
+ \sum_k \frac{1}{2} \mathbb{E} \left( \left[ \left( X_k - m_k^* \right)^+ \right]^2 \right) 
= c
\end{cases}
\end{equation*}

Definitions from the paper:

\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ f_k \left( m_k^* \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \left( e_k \left( m_k^* \right) 
+ \frac{1}{2} g_k \left( m_k^* \right)  \right) 
= c
\end{cases}
\end{equation*}

In [4]:
def e(index, x):
    mu = Mu[index]
    
    return mu - x

In [5]:
def fourier_integral1d(cplx_integrand, j, x, alpha):
    import warnings
    warnings.filterwarnings('error')
    
    def real_integrand(u):
        return scipy.real(cplx_integrand(u, j, x, alpha))

    real_quad = quad(real_integrand, -numpy.inf, numpy.inf, epsrel=1e-4)[0]
    
    return real_quad

In [6]:
def f_fourier_integrand(u, j, x, alpha):
    i = 1j
    
    mu = Mu[j]
    sigma2 = Sigma[j, j]
    
    alpha_m_iu = alpha - i*u
    
    log_part = -alpha_m_iu * x
    log_part += mu * alpha_m_iu + .5*sigma2*(alpha_m_iu)**2
    
    tmp = numpy.exp(log_part)
    tmp /= (u + i*alpha)**2
    
    return tmp

def f(index, x):
    factor = -.5/numpy.pi
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand()
            integral = fourier_integral1d(f_fourier_integrand, index, x, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x = %s, alpha = %s" % (x, alpha)
    
    return factor * integral

In [7]:
def g_fourier_integrand(u, j, x, alpha):
    i = 1j
    
    mu = Mu[j]
    sigma2 = Sigma[j, j]
    
    alpha_m_iu = alpha - i*u
    
    log_part = -alpha_m_iu * x
    log_part += mu * alpha_m_iu + .5*sigma2*(alpha_m_iu)**2
    
    tmp = numpy.exp(log_part)    
    tmp /= (i * (u + alpha*i)**3)
    
    return tmp

def g(index, x):
    factor = +1./numpy.pi
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand(1)
            integral = fourier_integral1d(g_fourier_integrand, index, x, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x = %s, alpha = %s" % (x, alpha)
    
    return factor * integral

In [8]:
class WrapperFunc(object):
    def __init__(self, func, indexes):
        self._i = indexes
        self._f = func
        
    def __call__(self, *x):
        tmp_args = self._i + list(x)
        return self._f(*tmp_args)

## Put the variables and functions to the remote engines

In [9]:
import sys

sys.path.append("../../../lib")

In [10]:
from tchebychev import TchebychevFun

def init_funcs():
    from timeit import default_timer as timer
    
    funcs = dict()
    interpol_time = 0.
    
    for k in range(N):
        ek = WrapperFunc(e, [k])
        tic = timer()
        ek = TchebychevFun(ek, [-10], [10], [CHEB_NODES_NB])
        interpol_time += timer() - tic
        
        fk = WrapperFunc(f, [k])
        tic = timer()
        fk = TchebychevFun(fk, [-10], [10], [CHEB_NODES_NB])
        interpol_time += timer() - tic
        
        gk = WrapperFunc(g, [k])
        tic = timer()
        gk = TchebychevFun(gk, [-10], [10], [CHEB_NODES_NB])
        interpol_time += timer() - tic
    
        funcs[k] = {"e": ek, "f": fk, "g": gk}
        
    global ctp
    ctp += interpol_time
        
    return funcs

In [11]:
def prepare_parameterized_funcs(dict_funcs):
    res = dict()
        
    for k, v in dict_funcs.iteritems():
        for kk, vv in v.iteritems():
            if isinstance(vv, TchebychevFun):
                key = "%s%s" % (kk, k)
                res[key] = (vv, k)
        
    return res

In [12]:
def run_par_func(param):
    f = param[0]
    vm = []
    for i_m in param[1:]:
        vm.append(M[i_m])
    
    return f(*vm)

In [13]:
funcs = init_funcs()
flatten_funcs = prepare_parameterized_funcs(funcs)

In [14]:
from timeit import default_timer as timer

In [15]:
from scipy.optimize import fsolve

def optimize(c, **kwargs):
    global N
    
    __results = {key: dict() for key in "efg"}
        
    def fill_results(lbl, res):
        fname = lbl[0]
        k = int(lbl[-1])
        
        if len(lbl) == 3:
            j = int(lbl[1])
            if k not in __results[fname]:
                __results[fname][k] = dict()
                
            __results[fname][k][j] = res
        else:
            __results[fname][k] = res
    
    def objective(v_x):
        _lambda = v_x[-1]
        
        # Local version
        global M
        M = v_x
        f_res = map(run_par_func, flatten_funcs.values())
        
        for lbl, fx in zip(flatten_funcs.keys(), f_res):
            fill_results(lbl, fx)
            
        last = 0.
        res = numpy.zeros(N + 1)
        
        for k in range(N):
            res_k = 1 + __results["f"][k]
            res[k] = (_lambda * res_k - 1.)**2
            
            last += __results["e"][k] + .5 * __results["g"][k]
                    
        res[-1] = (last - c)**2
        return res
    
    guess = numpy.zeros(N + 1)
    if 'guess' in kwargs and kwargs['guess'] is not None:
        guess = kwargs.pop('guess')
        
    return fsolve(objective, guess, **kwargs) 

In [16]:
c = 1.

guess = numpy.ones(N+1)

In [17]:
tic = timer()
optim_result = optimize(c, guess=guess, maxfev=1000, full_output=True)
toc_m_tic = timer() - tic

print optim_result[-1]

The solution converged.


In [18]:
m_star = optim_result[0][:-1]
lb_star = optim_result[0][-1]
cto = toc_m_tic

print "Time of optimization: %.2f sec" % (ctp)
print "m star = %s" % m_star
print "lambda star = %s" % lb_star

Time of optimization: 0.57 sec
m star = [-0.17267698 -0.17267698]
lambda star = 0.660130665013


In [19]:
l_N = optim_result[1]["nfev"]

print "l(2) = %i" % l_N
print "function evaluated at the output = %s" % optim_result[1]["fvec"]

l(2) = 49
function evaluated at the output = [  1.48240490e-17   1.48183566e-17   2.30484341e-17]


### Just need to copy the results in Excel:

In [20]:
print "%f \n %f \n %i \n %.2f \n %.2f" % (m_star[0], lb_star, l_N, ctp, cto)

-0.172677 
 0.660131 
 49 
 0.57 
 0.09
